In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import  Dense, Dropout
import keras_nlp

import tensorflow_text as text
import tensorflow_hub as hub

from sklearn.model_selection import train_test_split

/Users/mojc/code/essay_scoring/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mojc/code/essay_scoring/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("learning-agency-lab-automated-essay-scoring-2/train.csv")

X_train, X_val, y_train, y_val = train_test_split(data['full_text'], data['score'], test_size=0.2, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape )

(13845,) (3462,) (13845,) (3462,)


In [3]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_small_en_uncased", trainable=True)
encoder_inputs = preprocessor(text_input)

encoder = keras_nlp.models.BertBackbone.from_preset("bert_small_en_uncased", load_weights=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

2024-09-19 11:59:00.351810: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-09-19 11:59:00.351832: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-09-19 11:59:00.351838: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-09-19 11:59:00.351855: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-19 11:59:00.351866: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
l = tf.keras.layers.Dense(16, activation='relu', name='h1')(pooled_output)
# l = tf.keras.layers.Dense(64, activation='relu',name='h2')(l)
l = tf.keras.layers.Dense(32, activation='relu', name='h3')(l)
l = tf.keras.layers.Dense(1,activation='relu',name='output')(l)

model=tf.keras.Model(inputs=[text_input],outputs=[l])
model.compile(optimizer='adam',
              loss='MSE',
              metrics=['accuracy'])
model.summary()
# Non-trainable params: 0 (0.00 B)?

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_preprocessor   │ [(None, 512),     │          0 │ input_layer[0][0] │
│ (BertPreprocessor)  │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 512),     │ 28,763,648 │ bert_preprocesso… │
│ (BertBackbone)      │ (None, 512, 512)] │            │ bert_preprocesso… │
│                     │                   │            │ bert_preprocesso… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ h1 (Dense)          │ (None, 16)        │      8,208 │ bert_backbone[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ h3 (Dense)          │ (None, 32)        │        544 │ h1[0][0]          │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ h3[0][0]          │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,772,433 (109.76 MB)

 Trainable params: 28,772,433 (109.76 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val, y_val))
model.evaluate(X_val, y_val)
# accuracy: 0.0777 - loss: 1.1673

Epoch 1/5


2024-09-19 11:59:04.976754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


433/433 ━━━━━━━━━━━━━━━━━━━━ 559s 1s/step - accuracy: 0.0676 - loss: 1.4354 - val_accuracy: 0.0751 - val_loss: 1.1352
Epoch 2/5
433/433 ━━━━━━━━━━━━━━━━━━━━ 537s 1s/step - accuracy: 0.0697 - loss: 1.1082 - val_accuracy: 0.0751 - val_loss: 1.1229
Epoch 3/5
433/433 ━━━━━━━━━━━━━━━━━━━━ 535s 1s/step - accuracy: 0.0705 - loss: 1.0837 - val_accuracy: 0.0751 - val_loss: 1.1238
Epoch 4/5
433/433 ━━━━━━━━━━━━━━━━━━━━ 535s 1s/step - accuracy: 0.0710 - loss: 1.1228 - val_accuracy: 0.0751 - val_loss: 1.1054
Epoch 5/5
433/433 ━━━━━━━━━━━━━━━━━━━━ 551s 1s/step - accuracy: 0.0699 - loss: 1.1275 - val_accuracy: 0.0751 - val_loss: 1.1178
109/109 ━━━━━━━━━━━━━━━━━━━━ 40s 366ms/step - accuracy: 0.0777 - loss: 1.1413


[1.1178067922592163, 0.0751010999083519]

In [24]:
preds = model.predict(X_val)[:, 0]

109/109 ━━━━━━━━━━━━━━━━━━━━ 38s 352ms/step


In [29]:
print(np.mean(abs(preds - y_val)))
print(np.mean((np.round(preds, 0) - y_val)**2))

0.8415691465535377
1.1086077411900634


In [30]:
print(min(preds), max(preds))

2.8068044 2.8068066
